In [44]:
import numpy as np
import sympy as sp

In [45]:
labels = {
    'A': ("000", "1"),
    'B': ("001", "1"),
    'C': ("100", "1"),
    'D': ("101", "1"),
    'E': ("010", "0"),
    'F': ("011", "0"),
    'G': ("110", "0"),
    'H': ("111", "0")
}

In [46]:
# test
labels['A']

('000', '1')

In [43]:
# function to check if mask with ith cell dotted matches jth cell of sub
def doesMatch(mask, i, sub, j):
    m = len(mask)
    s = len(sub)
    if (i < 0 or j < 0 or j < i or j - i + m > s or i >= m or j >= s):
        # print(mask, ": ", i, j, m, s, "#", i < 0, j < 0, j < i, j - i + m >= s, i >= m, j >= s)
        # print(sub)
        return 0
    else:
        if (mask == sub[j - i: j - i + m]):
            return 1
        else:
            return 0


In [29]:
# test
doesMatch("0", 0, "0011", 2)

0

In [30]:
# function to decide if a cell is able to switch its strategy regarding transition rules
def decideIfTheCellCanFlip(left, center, right, transition_rules):
    n = transition_rules.shape[0]
    for i in range(n):
        if (transition_rules[i][0][0] == left and transition_rules[i][0][1] == center and transition_rules[i][0][2] == right and transition_rules[i][1][0] != center):
            return True
    return False

In [31]:
# general function to calculate probability of conversion of sub-config A to B regarding certain transition rules
def generalCalcProb(A, B, transition_rules = np.array([labels['A'], labels['B'], labels['F'], labels['C'], labels['G'], labels['H']])):
    a = len(A)
    b = len(B)
    if (a != b or A[0] != B[0] or A[a - 1] != B[b - 1]):
        return 0
    r, p = sp.symbols("r p")
    r = 1
    for i in range(1, a - 1):    # ignoring first and last cell
        if (A[i] == B[i]):    # must remain on its strategy
            if (decideIfTheCellCanFlip(A[i - 1], A[i], A[i + 1], transition_rules)):    # checking if the ith cell can flip
                r *= (1 - p)
            else:
                r *= 1
        else:
            if (decideIfTheCellCanFlip(A[i - 1], A[i], A[i + 1], transition_rules)):
                r *= p
            else:
                return 0
    return r


In [32]:
# test
print(generalCalcProb("011110", "010110"))

p*(1 - p)**3


In [33]:
def generateAllBinaryStrings(length):
    arr = [""] * 2 ** length
    form = '#0' + str(length + 2) + 'b'
    for i in range(2 ** length):
        arr[i] = format(i, form)[2:]
    return np.array(arr)


In [34]:
# test
print(generateAllBinaryStrings(3))

['000' '001' '010' '011' '100' '101' '110' '111']


In [35]:
# function to calculate the expected difference of number of a list of patterns in a sub-config
def calcExpectedDifference(patterns, sub_config, dot_index, transition_rules = np.array([labels['A'], labels['B'], labels['F'], labels['C'], labels['G'], labels['H']])):
    # patterns_max_length = np.max([len(i) for i in patterns[:, 0]])
    k = len(sub_config) - 2
    n = 2 ** k
    binary_strings = generateAllBinaryStrings(k)
    number_of_patterns = patterns.shape[0]
    expected_variation = [0] * number_of_patterns
    for j in range(number_of_patterns):
        #
        total_expectation = 0
        current_number_of_occurrence = doesMatch(patterns[j][0], int(patterns[j][1]), sub_config[1:-1], dot_index - 1)
        for i in range(n):
            total_expectation += generalCalcProb(sub_config, sub_config[0] + binary_strings[i] + sub_config[-1], transition_rules) * (doesMatch(patterns[j][0], int(patterns[j][1]), binary_strings[i], dot_index - 1) - current_number_of_occurrence)
        #
        expected_variation[j] = total_expectation
    return np.array(expected_variation)


In [36]:
# test
pats = np.array([["0", 0], ["11", 1], ["01", 1]])
tr = np.array([labels['E'], labels['F']])
calcExpectedDifference(pats, "0011", 2, tr)

array([p, 0, -p], dtype=object)

In [37]:
#
def getAllDifferencesOnExpectationWithGivenPatterns(patterns, dot_index, transition_rules = np.array([labels['A'], labels['B'], labels['F'], labels['C'], labels['G'], labels['H']])):
    pattern_max_length = np.max([len(i) for i in patterns[:, 0]])
    m = patterns.shape[0]

    # binary sub-config length should be equal with pattern_length + 2
    k = pattern_max_length + 2
    n = 2 ** k
    binary_sub_configs = generateAllBinaryStrings(k)
    diffs = [ [0] * m for i in range(n)]
    for j in range(n):
        expectations_of_current_sub_config = calcExpectedDifference(patterns, binary_sub_configs[j], dot_index, transition_rules)
        diffs[j] = expectations_of_current_sub_config
    
    return np.array(diffs).T


In [38]:
# pats = np.array([["0", 0], ["11", 1], ["01", 1]])
# tr = np.array([labels['E'], labels['F']])
# coefs = np.array([0, 1, 1])



p = sp.symbols('p')
c = -sp.floor(3 / p) - 1
b = -1
a = -2 * c + 2

pats = np.array([["101", 1], ["011", 1], ["110", 1], ["111", 1], ["010", 1]])
tr = np.array([labels['B'], labels['E'], labels['F']])
coefs = np.array([c, a + b, a, a, a])



# pats = np.array([["0", 0], ["010", 1], ["01011", 3]])
# coefs = np.array([3, 3, 1])
# tr = np.array([labels['B'], labels['E']])



# pats = np.array([["0", 0], ["010", 1], ["011", 1], ["11", 1]])
# coefs = np.array([0, 6, 5, 5])
# tr = np.array([labels['B'], labels['E'], labels['F'], labels['G']])
# pats = np.array([["1", 0], ["010", 2]])
# coefs = np.array([5, 1])
# tr = np.array([labels['B'], labels['E'], labels['F'], labels['G']])

x = getAllDifferencesOnExpectationWithGivenPatterns(pats, 2, tr)
y = np.dot(coefs, x)
y

array([0, 0, p**2*(2*floor(3/p) + 4) + p*(1 - p)*(2*floor(3/p) + 3),
       p**2*(2*floor(3/p) + 4) + p*(1 - p)*(2*floor(3/p) + 3),
       p*(1 - p)*(2*floor(3/p) + 4) + (-p**2 - 2*p*(1 - p))*(2*floor(3/p) + 4),
       p*(1 - p)*(2*floor(3/p) + 4) + (-p**2 - 2*p*(1 - p))*(2*floor(3/p) + 4),
       p**2*(-floor(3/p) - 1) + p*(1 - p)*(2*floor(3/p) + 4) + (-p**2 - 2*p*(1 - p))*(2*floor(3/p) + 3),
       p**2*(-floor(3/p) - 1) + p*(1 - p)*(2*floor(3/p) + 4) + (-p**2 - 2*p*(1 - p))*(2*floor(3/p) + 3),
       0, p*(1 - p)*(-floor(3/p) - 1),
       (-p**2 - 2*p*(1 - p))*(-floor(3/p) - 1),
       (-p**2 - 2*p*(1 - p))*(-floor(3/p) - 1), 0, 0,
       p*(2*floor(3/p) + 3) - p*(2*floor(3/p) + 4),
       p*(2*floor(3/p) + 3) - p*(2*floor(3/p) + 4), 0, 0,
       p**2*(2*floor(3/p) + 4) + p*(1 - p)*(2*floor(3/p) + 3),
       p**2*(2*floor(3/p) + 4) + p*(1 - p)*(2*floor(3/p) + 3),
       -p*(2*floor(3/p) + 4), -p*(2*floor(3/p) + 4),
       -p*(2*floor(3/p) + 3), -p*(2*floor(3/p) + 3), 0,
       p*(-f

In [39]:
for i in y:
    print(i)

0
0
p**2*(2*floor(3/p) + 4) + p*(1 - p)*(2*floor(3/p) + 3)
p**2*(2*floor(3/p) + 4) + p*(1 - p)*(2*floor(3/p) + 3)
p*(1 - p)*(2*floor(3/p) + 4) + (-p**2 - 2*p*(1 - p))*(2*floor(3/p) + 4)
p*(1 - p)*(2*floor(3/p) + 4) + (-p**2 - 2*p*(1 - p))*(2*floor(3/p) + 4)
p**2*(-floor(3/p) - 1) + p*(1 - p)*(2*floor(3/p) + 4) + (-p**2 - 2*p*(1 - p))*(2*floor(3/p) + 3)
p**2*(-floor(3/p) - 1) + p*(1 - p)*(2*floor(3/p) + 4) + (-p**2 - 2*p*(1 - p))*(2*floor(3/p) + 3)
0
p*(1 - p)*(-floor(3/p) - 1)
(-p**2 - 2*p*(1 - p))*(-floor(3/p) - 1)
(-p**2 - 2*p*(1 - p))*(-floor(3/p) - 1)
0
0
p*(2*floor(3/p) + 3) - p*(2*floor(3/p) + 4)
p*(2*floor(3/p) + 3) - p*(2*floor(3/p) + 4)
0
0
p**2*(2*floor(3/p) + 4) + p*(1 - p)*(2*floor(3/p) + 3)
p**2*(2*floor(3/p) + 4) + p*(1 - p)*(2*floor(3/p) + 3)
-p*(2*floor(3/p) + 4)
-p*(2*floor(3/p) + 4)
-p*(2*floor(3/p) + 3)
-p*(2*floor(3/p) + 3)
0
p*(-floor(3/p) - 1)
-p*(-floor(3/p) - 1)
-p*(-floor(3/p) - 1)
0
0
0
0
